In [ ]:
%pip install cohere

In [ ]:
%pip install guardrails-ai pydantic typing openai rich

In [ ]:
import guardrails as gd

In [ ]:
from pydantic import BaseModel, Field
from guardrails.validators import ValidLength, TwoWords, ValidRange
from guardrails.datatypes import Date
from typing import List

prompt = """
Generate a dataset of fake user orders. Each row of the dataset should be valid. The format should not be a list, it should be a JSON object.
${gr.complete_json_suffix}

an example of output may look like this:
{
	"user_orders": [{                                                                                                   │ │
        "user_id": 1,
        "user_name": "John Mcdonald",
    	"num_orders": 6
    }]
}

"""

class Order(BaseModel):
	user_id: int = Field(description="The user's id.", validators=[("1-indexed", "noop")])
	user_name: str = Field(
		description="The user's first name and last name",
		validators=[TwoWords()]
	)

	num_orders: int = Field(
		description="The number of orders the user has placed",
		validators=[ValidRange(0, 50)]
	)
	


class Orders(BaseModel):
	user_orders: List[Order] = Field(
		description="Generate a list of users and how many orders they have placed in the past.",
		validators=[ValidLength(10, 10, on_fail="noop")]
	)

In [ ]:
import cohere

co = cohere.Client(api_key='<Cohere_API_KEY>')

In [ ]:
guard = gd.Guard.from_pydantic(output_class=Orders, prompt=prompt)

In [ ]:
raw_llm_response, validated_response = guard(
	co.generate,
	model="command",
	max_tokens=1024,
	temperature=0.3
)

In [ ]:
from rich import print

guard.guard_state.most_recent_call.tree